***Bienvenue sur le Notebook qui réalise de la classification supervisée sur des données de la forme:

**Texte descriptif,0
Autre texte,1
Encore un autre texte,0

**(Ici 1 veut dire que le texte parle d'un endroit aux Etats Unis, 0 sinon) 
 
**Le but de ce Notebook est de prédire si le texte en question parle d'un endroit aux Etats-Unis ou d'un endroit ailleurs sur la planète

**Pour ce faire, nous créons un dataframe qui lis le fichier contenant les données et qui met les textes dans la première colonne que l'on nommera data (variables prédictives), et qui met les 0 et 1 dans une colonne nommée target (variable à prédire)

**Nous allons également compter le nombre de textes parlant d'un endroit aux Etats Unis et les autres, afin de nous assurer qu'ils sont représentés dans des proportions à peu près égales


In [54]:
import pandas as pd
df=pd.read_csv("myfile.csv",sep=',', header = None, names = ['data', 'target'])
print(df.head())
list(df.columns)
cptUSA=0
cptOthers=0
for i in df.target:
    if i==1:
        cptUSA=cptUSA+1
    else:
        cptOthers= cptOthers+1
print("dans le csv de départ, il y a ",cptUSA," photos prises aux USA et ",cptOthers, "photos prises ailleurs") 

                                                data  target
0                               Estes Park Colorado        1
1  The most beautiful mountain range in the Dolom...       0
2       Sunset burns over Reflections lake in Alaska       1
3  Bloodmoon Canyon - Original composited Lunar E...       0
4               Sunset reflections in Wasilla Alaska       1
dans le csv de départ, il y a  520  photos prises aux USA et  762 photos prises ailleurs


**Vectorisation des textes pour qu'ils deviennent utilisables par les modèles de clasification par la suite

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer  
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df.data)

**X -> variables prédictive, y -> variable à prédire

In [56]:
X = vectors.toarray()

y = df.target

**On découpe le jeu de données pour avoir 30% du jeu réservé à la validation

In [57]:
from sklearn.model_selection import train_test_split 

validation_size=0.3 #30% du jeu de données pour le test

testsize= 1-validation_size
seed=30
X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)

**On utilise la validation croisée pour faire le test de validation sur une partie différente du jeu de de données dix fois de suite et on fait ensuite la moyenne des accuracy trouvées

In [58]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from time import time

seed=7
k_fold = KFold(n_splits=10, shuffle=True, random_state=seed)
clf = GaussianNB()

scoring = 'accuracy'
t0 = time()
score = cross_val_score(clf, X, y, cv=k_fold, scoring=scoring)
print("Réalisé en %0.3fs" % (time() - t0))

print('Les différentes accuracy pour les 10 évaluations sont : \n',
      score,'\n')
print ('Accuracy moyenne : ',score.mean(), 
       ' standard deviation', score.std())

Réalisé en 0.501s
Les différentes accuracy pour les 10 évaluations sont : 
 [0.98449612 0.99224806 0.9765625  0.9921875  0.9765625  0.9921875
 0.984375   0.984375   0.9921875  0.96875   ] 

Accuracy moyenne :  0.9843931686046512  standard deviation 0.007818650022855016


**On teste la méthode de classification linéaire SGDC et on affiche la matrice de confusion et le rapport de classification

In [59]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from time import time
from sklearn.metrics import classification_report

pipeline = Pipeline([('vect', TfidfVectorizer()),
                ('clf', SGDClassifier(loss='hinge', 
                                      penalty='l2',
                                      alpha=1e-3, 
                                      random_state=42, 
                                      max_iter=5, tol=None)),
               ])




X=df.data
y=df.target


X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)


t0 = time()
pipeline.fit(X_train, y_train)
print("Fit réalisé en %0.3fs" % (time() - t0))

t0 = time()
result = pipeline.predict(X_test)
print("Prédiction réalisée en %0.3fs" % (time() - t0))

print('\n accuracy:',accuracy_score(result, y_test),'\n')




conf = confusion_matrix(y_test, result)
print ('\n matrice de confusion \n',conf)



print ('\n',classification_report(y_test, result))

Fit réalisé en 0.013s
Prédiction réalisée en 0.026s

 accuracy: 0.8697104677060133 


 matrice de confusion 
 [[474  48]
 [ 69 307]]

               precision    recall  f1-score   support

           0       0.87      0.91      0.89       522
           1       0.86      0.82      0.84       376

    accuracy                           0.87       898
   macro avg       0.87      0.86      0.87       898
weighted avg       0.87      0.87      0.87       898



**On utilise une GridSearchCV pour tester toutes les combinaisons de paramètres des modèles linéaires et d'arbre de décision, puis on choisit le meilleurs résultat pour chaque modèle et on les compare. (ici, le classifieur linéaire donne une meilleure accuracy)

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from time import time
from sklearn.svm import SVC
import pickle


# Specification des pipelines
# programmation à optimiser par une fonction :)
pipeline_SGDC = Pipeline([('tfidf', TfidfVectorizer()),
                    ('clf', SGDClassifier())])


parameters_SGDC = [
    {'clf__max_iter': (8,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet')}
]

pipeline_DT = Pipeline([('tfidf', TfidfVectorizer()),
                   ('clf', DecisionTreeClassifier())])


#param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 5, 8, 10]
parameters_DT = [
    {'clf__min_samples_leaf': param_range,
        'clf__criterion': ['gini', 'entropy'],
        'clf__max_depth': param_range,
        'clf__min_samples_split': param_range[1:]}
]




X=df.data
y=df.target


X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)

# Creation des GridSearchCV avec les pipelines spécifiques

gs_SGDC = GridSearchCV(pipeline_SGDC, 
                       parameters_SGDC, 
                       cv=3,
                       n_jobs=-1, 
                       scoring='accuracy')


gs_DT = GridSearchCV(pipeline_DT, 
                     parameters_DT, 
                     cv=3,
                     n_jobs=-1, 
                     scoring='accuracy')



grids = [gs_SGDC, gs_DT]
grid_dict={0:'Linear classifiers', 1:'Decision Tree'}

best_acc = 0.0
best_clf = 0.0
best_gs = ''

for idx,gs in enumerate(grids):
    print('\nClassifier: %s' % grid_dict[idx])
    t0 = time()
    gs.fit(X_train, y_train)
    print("Fit réalisé en %0.3fs" % (time() - t0))

    print('Meilleurs paramètres : %s' % gs.best_params_)

    print("Meilleur score d'accuracy sur l'entrainement: %.3f" % gs.best_score_)
    # Prediction sur le jeu de test avec les meilleurs paramètres
    t0 = time()
    result = gs.predict(X_test)
    print("Prédiction réalisée en %0.3fs" % (time() - t0))
    
    print("Score d'accuracy pour les meilleurs paramètres sur jeu de test : %.3f"  % accuracy_score(y_test, result))

    print ('\n matrice de confusion \n',confusion_matrix(y_test, result))

    print ('\n',classification_report(y_test, result))
    
    #Modele avec la meilleure accuracy sur le jeu de test
    if accuracy_score(y_test, result) > best_acc:
        best_acc = accuracy_score(y_test, result)
        best_gs = gs
        best_clf = idx
        
        
        
print('\nClassifier avec la meilleur accuracy sur le jeu de test\n',
      grid_dict[best_clf])        


Classifier: Linear classifiers
Fit réalisé en 0.109s
Meilleurs paramètres : {'clf__alpha': 1e-05, 'clf__max_iter': 8, 'clf__penalty': 'l2'}
Meilleur score d'accuracy sur l'entrainement: 0.556
Prédiction réalisée en 0.002s
Score d'accuracy pour les meilleurs paramètres sur jeu de test : 0.667

 matrice de confusion 
 [[7 3]
 [4 7]]

               precision    recall  f1-score   support

           0       0.64      0.70      0.67        10
           1       0.70      0.64      0.67        11

    accuracy                           0.67        21
   macro avg       0.67      0.67      0.67        21
weighted avg       0.67      0.67      0.67        21


Classifier: Decision Tree
Fit réalisé en 1.497s
Meilleurs paramètres : {'clf__criterion': 'gini', 'clf__max_depth': 5, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 5}
Meilleur score d'accuracy sur l'entrainement: 0.778
Prédiction réalisée en 0.002s
Score d'accuracy pour les meilleurs paramètres sur jeu de test : 0.524

 matri

**On recommence le processus pour obtenir le modèle du classifieurs linéaire entraîné puis on le compresse avec pickle

In [61]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from time import time
from sklearn.metrics import classification_report


#Recupération des données pour l'exemple
#et partir proprement
#categories = ['alt.atheism', 'talk.religion.misc',
#             'rec.sport.hockey','comp.graphics', 'sci.space']
#
#news = fetch_20newsgroups(subset='all',
#                         categories=categories)

df=pd.read_csv("myfile.csv",sep=',', header = None, names = ['data', 'target'])




pipeline = Pipeline([('vect', TfidfVectorizer()),
                ('clf', SGDClassifier(loss='hinge', 
                                      penalty='l2',
                                      alpha=1e-05, 
                                      random_state=42, 
                                      max_iter=5, 
                                      tol=None)),
               ])




X=df.data
y=df.target


X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)


t0 = time()
print ("Lancement du fit \n")
pipeline.fit(X_train, y_train)
print("Fit réalisé en %0.3fs" % (time() - t0))

t0 = time()
print ("Lancement de la prédiction \n")
result = pipeline.predict(X_test)
print("Prédiction réalisée en %0.3fs" % (time() - t0))

print('\n accuracy:',accuracy_score(result, y_test),'\n')

conf = confusion_matrix(y_test, result)
print ('\n matrice de confusion \n',conf)



print ('\n',classification_report(y_test, result))

print("\nSauvegarde du pipeline grid search") 
filename = 'thebestone1.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

Lancement du fit 

Fit réalisé en 0.013s
Lancement de la prédiction 

Prédiction réalisée en 0.025s

 accuracy: 0.8741648106904232 


 matrice de confusion 
 [[466  56]
 [ 57 319]]

               precision    recall  f1-score   support

           0       0.89      0.89      0.89       522
           1       0.85      0.85      0.85       376

    accuracy                           0.87       898
   macro avg       0.87      0.87      0.87       898
weighted avg       0.87      0.87      0.87       898


Sauvegarde du pipeline grid search


**On charge le modèle entraîné ci dessus et on teste les prédictions sur des données que le modèle n'a jamais vu, ni en entraînement ni en validation

In [63]:
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print ("Chargement du modèle \n")
filename = 'thebestone1.pkl'
clf_loaded = pickle.load(open(filename, 'rb'))


print ("A partir d'un nouveau texte\n")
print ("Utilisation d'un texte\n")

df=pd.read_csv("newfile.csv",sep=',', names = ['data', 'target'])

print(df)

print ("Sélection aléatoire de 15 documents \n")
from random import randint
samples=[]
samples_result=[]
sample_new=[]

for i in range(1,15):
    val=randint(1,15)
    sample_new.append(val)
    samples.append(df.data[val])
    samples_result.append(df.target[val])
    
print ("Prédiction des news séléctionnées\n")    
    
 

result = clf_loaded.predict(samples)

print ("Valeurs réelles vs. valeurs prédites\n") 
for i in range(len(result)):
    print ("texte ",sample_new[i], 
           "\t réelle ", 
           samples_result[i], 
           " prédite ",
           result [i])

Chargement du modèle 

A partir d'un nouveau texte

Utilisation d'un texte

                                                 data  target
0                                Estes Park Colorado        1
1   The most beautiful mountain range in the Dolom...       0
2        Sunset burns over Reflections lake in Alaska       1
3   Bloodmoon Canyon - Original composited Lunar E...       0
4                Sunset reflections in Wasilla Alaska       1
5                                   Kootenai Falls MT       1
6                          Sunset Over Lake Superior        0
7         Devils Punchbowl Waterfall Arthur's Pass NZ       0
8   The Ghost Mountains Kamchatka Photo Isabella T...       0
9              Hidden cove at Oswald West Park Oregon       1
10                                  Lake Matheson NZ        0
11       Black basalt waterfall at Svartifoss Iceland       0
12                      Port Orford Oregon at sunset        1
13                    Rattlesnake Lake North Bend WA    

**Ici on voit que Presque toutes les valeurs prédites sont égales au valeurs réelles, comme le laissait penser les accuracy trouvées plus haut